# **1. Import Library**

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [5]:
data = pd.read_csv('data_with_clusters.csv')
data.head()

,Artist,Track,Album,Album_type,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,...,Comments,Licensed,official_video,Stream,EnergyLiveness,most_playedon,Cluster,PCA1,PCA2,DBSCAN_Cluster
0,686,4982,2637,0,1.195758,0.325870,0.215386,0.715524,-0.989238,-0.278033,...,0.748338,2,2,3.744920,-0.975691,0,2,3.639427,3.098091,-1
1,686,12384,7751,0,0.337753,0.316536,0.401618,-0.593096,-0.714728,-0.286519,...,0.021741,2,2,0.732154,2.432865,0,2,1.745894,-0.450165,-1
2,686,10329,6933,2,0.452557,1.343283,0.807923,-0.396981,-0.869913,-0.047836,...,-0.101730,2,2,-0.287107,0.677596,0,2,1.334604,-1.246265,-1
3,686,10894,7751,0,0.416303,0.484549,0.402696,-0.630536,-1.018405,2.338837,...,0.148465,2,2,1.246198,1.549555,0,2,1.216986,0.795395,-1
4,686,2897,4139,0,0.259203,0.274533,-0.204499,0.662038,-0.930030,-0.290067,...,0.675226,2,2,1.999630,1.159920,1,2,3.418022,2.891852,-1


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [6]:
# Memisahkan fitur dan target
X = data.drop(columns=['Cluster'])  # Fitur adalah semua kolom kecuali 'Cluster'
y = data['Cluster']  # Target adalah kolom 'Cluster'

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Menampilkan ukuran data latih dan data uji
print(f"Ukuran Data Latih: {X_train.shape[0]} baris")
print(f"Ukuran Data Uji: {X_test.shape[0]} baris")

Ukuran Data Latih: 16475 baris
Ukuran Data Uji: 4119 baris


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [7]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(random_state=42)


pada pelatihan kali ini saya menggunakan algoritma Randomforestsetelah melakukan beberapa pertimbangan berdasarkan fitur yang ada

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [8]:
# Evaluasi model awal (sebelum tuning)
rf_model.fit(X_train, y_train)  # Melatih model awal
y_pred_initial = rf_model.predict(X_test)  # Prediksi data uji
accuracy_initial = accuracy_score(y_test, y_pred_initial)  # Menghitung akurasi model awal
print(f"Akurasi Model Awal: {accuracy_initial * 100:.2f}%")
print("\nConfusion Matrix (Awal):")
print(confusion_matrix(y_test, y_pred_initial))
print("\nClassification Report (Awal):")
print(classification_report(y_test, y_pred_initial))

# Grid Search untuk menemukan hyperparameter terbaik
param_grid = {
    'n_estimators': [50, 100, 200],  # Jumlah pohon dalam forest
    'max_depth': [None, 10, 20, 30],  # Kedalaman maksimum pohon
    'min_samples_split': [2, 5, 10],  # Jumlah minimum sampel untuk split node
    'min_samples_leaf': [1, 2, 4],  # Jumlah minimum sampel di leaf node
    'bootstrap': [True, False]  # Apakah bootstrap digunakan
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

Akurasi Model Awal: 99.03%

Confusion Matrix (Awal):
[[ 439    5   10]
 [   7  873   17]
 [   0    1 2767]]

Classification Report (Awal):
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       454
           1       0.99      0.97      0.98       897
           2       0.99      1.00      0.99      2768

    accuracy                           0.99      4119
   macro avg       0.99      0.98      0.98      4119
weighted avg       0.99      0.99      0.99      4119



Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

### GridSearch

In [ ]:
# Latih model dengan GridSearchCV
grid_search.fit(X_train, y_train)

# Menampilkan hyperparameter terbaik
print(f"Best Hyperparameters (Grid Search): {grid_search.best_params_}")

# Evaluasi model terbaik dari GridSearchCV
best_model_grid = grid_search.best_estimator_  # Model terbaik hasil GridSearchCV
y_pred_grid = best_model_grid.predict(X_test)  # Prediksi data uji dengan model terbaik
accuracy_grid = accuracy_score(y_test, y_pred_grid)  # Menghitung akurasi
print(f"Akurasi Model (Grid Search): {accuracy_grid * 100:.2f}%")
print("\nConfusion Matrix (Grid Search):")
print(confusion_matrix(y_test, y_pred_grid))
print("\nClassification Report (Grid Search):")
print(classification_report(y_test, y_pred_grid))

# Randomized Search untuk menemukan hyperparameter terbaik
param_dist = {
    'n_estimators': randint(50, 200),  # Distribusi jumlah pohon
    'max_depth': [None, 10, 20, 30, 40],  # Kedalaman maksimum pohon
    'min_samples_split': randint(2, 10),  # Distribusi minimum sampel untuk split node
    'min_samples_leaf': randint(1, 4),  # Distribusi minimum sampel di leaf node
    'bootstrap': [True, False]  # Apakah bootstrap digunakan
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist,
                                   n_iter=100, cv=5, n_jobs=-1, verbose=2, scoring='accuracy', random_state=42)

# Latih model dengan RandomizedSearchCV
random_search.fit(X_train, y_train)

# Menampilkan hyperparameter terbaik dari RandomizedSearchCV
print(f"Best Hyperparameters (Randomized Search): {random_search.best_params_}")

# Evaluasi model terbaik dari RandomizedSearchCV
best_model_random = random_search.best_estimator_  # Model terbaik hasil RandomizedSearchCV
y_pred_random = best_model_random.predict(X_test)  # Prediksi data uji dengan model terbaik
accuracy_random = accuracy_score(y_test, y_pred_random)  # Menghitung akurasi
print(f"Akurasi Model (Randomized Search): {accuracy_random * 100:.2f}%")
print("\nConfusion Matrix (Randomized Search):")
print(confusion_matrix(y_test, y_pred_random))
print("\nClassification Report (Randomized Search):")
print(classification_report(y_test, y_pred_random))

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [2]:
# Analisis hasil evaluasi
print("\nAnalisis Hasil Evaluasi Model Klasifikasi:")

# Bandingkan hasil evaluasi sebelum dan setelah tuning
print(f"Akurasi Sebelum Tuning: {accuracy_initial * 100:.2f}%")
print(f"Akurasi Setelah Tuning (Grid Search): {accuracy_grid * 100:.2f}%")
print(f"Akurasi Setelah Tuning (Randomized Search): {accuracy_random * 100:.2f}%")

# Identifikasi kelemahan model
report_initial = classification_report(y_test, y_pred_initial, output_dict=True)
report_grid = classification_report(y_test, y_pred_grid, output_dict=True)
report_random = classification_report(y_test, y_pred_random, output_dict=True)

for cls in report_initial.keys():
    if isinstance(report_initial[cls], dict):
        print(f"\nKelas {cls}:")
        print(f"  Precision Sebelum Tuning: {report_initial[cls]['precision']:.2f}")
        print(f"  Recall Sebelum Tuning: {report_initial[cls]['recall']:.2f}")
        print(f"  Precision Setelah Tuning (Grid Search): {report_grid[cls]['precision']:.2f}")
        print(f"  Recall Setelah Tuning (Grid Search): {report_grid[cls]['recall']:.2f}")
        print(f"  Precision Setelah Tuning (Randomized Search): {report_random[cls]['precision']:.2f}")
        print(f"  Recall Setelah Tuning (Randomized Search): {report_random[cls]['recall']:.2f}")

# Identifikasi overfitting atau underfitting
if accuracy_initial > accuracy_grid or accuracy_initial > accuracy_random:
    print("\nModel mungkin mengalami overfitting pada data latih sebelum tuning.")
else:
    print("\nModel menunjukkan peningkatan performa setelah tuning.")

# Rekomendasi tindakan lanjutan
print("\nRekomendasi Tindakan Lanjutan:")
print("1. Mengumpulkan lebih banyak data untuk kelas minoritas jika precision atau recall rendah.")
print("2. Mencoba algoritma lain seperti Gradient Boosting atau XGBoost untuk membandingkan performa.")
print("3. Melakukan teknik resampling seperti oversampling atau undersampling untuk mengatasi ketidakseimbangan data.")


Analisis Hasil Evaluasi Model Klasifikasi:


NameError: name 'accuracy_initial' is not defined


Analisis Hasil Evaluasi Model Klasifikasi:


NameError: name 'accuracy_initial' is not defined

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.